# 

# "Nombres parfaits, nombres amicaux et nombres sociables"
> "On se propose dans cette étude d'exhiber des nombres parfaits, des nombres amicaux et des nombres sociables."

- toc: true
- branch: master
- badges: true
- comments: true
- author: Cédric Bohnert
- categories: [jupyter]

### Quelques définitions

Un nombre parfait est un nombre $n$ dont la somme de ses diviseurs propres (donc différents de $n$) est égale à $n$.

Un couple de nombre amicaux $(m,n)$ est un couple constitué de deux entiers strictement positifs distincts tels que la somme des diviseurs propre de chacun est égal à l'autre.

Un nombre sociable est un élément d'une chaîne sociable qui est une séquence $(a_1, \dots, a_n)$ d'entiers strictement positifs tels que chacun soit égal à la somme des diviseurs propres du précédent (cycliquement).

### Recherche de nombres parfaits par méthode naive

On recherchera les diviseurs propres d'un entier strictement positif par exhaustivité.

On testera la somme des diviseurs propres ainsi obtenus pour vérifier si un nombre $n$ est parfait.

On recherchera tous les nombres parfaits inférieurs ou égaux à un entier $N$ donné.

In [1]:
def getDivisors(n):
    """Returns a list of all positive divisors of integer n>0"""
    divisors = []
    for i in range(1, n+1):
        if n % i == 0:
            divisors.append(i)
    return divisors
def isPerfect(n):
    """Returns True of n is a perfect number"""
    return (sum(getDivisors(n))-n) == n

def getPerfectNumbers(N):
    """Returns a list of all perfect numbers less than N>0"""
    perfects = []
    for i in range(1,N+1):
        if isPerfect(i):
            perfects.append(i)
    return perfects

In [2]:
getDivisors(124)

[1, 2, 4, 31, 62, 124]

In [3]:
isPerfect(124)

False

In [4]:
isPerfect(496)

True

Recherchons les nombres parfaits inférieurs à 100000 et mesurons le temps de calcul :

In [5]:
%time perfects = getPerfectNumbers(100000)

CPU times: user 3min 52s, sys: 21.9 ms, total: 3min 52s
Wall time: 3min 52s


In [6]:
perfects

[6, 28, 496, 8128]

Les nombres parfaits inférieurs à 100000 sont 6, 28, 496 et 8128.

Le temps de calcul est assez long pour cette valeur et nous verrons ultérieurement que l'on peut améliorer la rapidité de ce calcul.

### Un somme particulière

Calculons la somme des inverses des diviseurs des quelques premiers nombres parfaits :

In [7]:
for i in perfects:
    s = sum([1/d for d in getDivisors(i)])
    print("n = ", i, ": ", s)

n =  6 :  2.0
n =  28 :  2.0
n =  496 :  2.0
n =  8128 :  2.0


On observe que cette somme vaut 2 pour les quatre premiers nombres parfaits.

Conjecturons que ce résultat est valide quelque soit le nombre parfait et démontrons le :

Soient $n$ un nombre parfait et $d_i$, $i=1, \dots , k$ ses $k$ diviseurs, rangés dans l'ordre croissant et où $d_1=1$ et $d_k=n$.

Notons $S = \sum_{i=1}^k \frac{1}{d_i}$.

En multipliant $S$ par $n$ on obtient aisement, $nS = n ( \sum_{i=1}^k \frac{1}{d_i}) = \sum_{i=1}^k \frac{n}{d_i}$.

Cette dernière somme n'est autre que la somme des diviseurs de $n$ rangés dans l'ordre décroissant.

Comme $n$ est un nombre parfait, la somme de ses diviseurs vaut $2n$. 

D'où le résultat : $S=2$.

### Recherche de nombres amicaux par méthode naive

On recherchera les nombres amicaux $(m,n)$, tels que $m \leq N$, pour un entier $N$ donné :

In [8]:
def getFriends(N):
    """Returns a list of all friendly numbers (m,n) for a given N such that  m<=N"""
    friends = []
    temp = []
    for m in range(1,N+1):
        if not m in temp:
            s1 = sum(getDivisors(m))-m
            s2 = sum(getDivisors(s1))-s1
            if m==s2 and m!=s1:
                friends.append((m, s1))
                temp.append(s1)
    return friends

In [9]:
%time getFriends(100000)

CPU times: user 6min 21s, sys: 4.02 ms, total: 6min 21s
Wall time: 6min 21s


[(220, 284),
 (1184, 1210),
 (2620, 2924),
 (5020, 5564),
 (6232, 6368),
 (10744, 10856),
 (12285, 14595),
 (17296, 18416),
 (63020, 76084),
 (66928, 66992),
 (67095, 71145),
 (69615, 87633),
 (79750, 88730)]

Nous déterminons 13 paires de nombres amicaux en plus de 5min de temps de calcul.

In [10]:
%time getFriends(200000)

CPU times: user 26min 3s, sys: 264 ms, total: 26min 3s
Wall time: 26min 6s


[(220, 284),
 (1184, 1210),
 (2620, 2924),
 (5020, 5564),
 (6232, 6368),
 (10744, 10856),
 (12285, 14595),
 (17296, 18416),
 (63020, 76084),
 (66928, 66992),
 (67095, 71145),
 (69615, 87633),
 (79750, 88730),
 (100485, 124155),
 (122265, 139815),
 (122368, 123152),
 (141664, 153176),
 (142310, 168730),
 (171856, 176336),
 (176272, 180848),
 (185368, 203432),
 (196724, 202444)]

Cette fois, nous déterminons 22 couples de nombres amicaux mais en plus de 20min de temps de calcul.

### Recherche de nombres sociables par méthode améliorée

On recherchera les chaînes sociables ne faisant intervenir que des nombres inférieurs à 1000000 et contenant au moins un nombre inférieur ou égal à 15000.

Notons que la somme de tous les diviseurs de $n$ (y compris $n$) s'exprime :

$$s(n)= \prod_{i=1}^k \frac{p_i^{\alpha_i+1}-1}{p_i-1}$$

où les $p_i$ et $\alpha_i$ sont les termes de la décomposition en facteurs premiers de $n$.

Utiliser cette expression nous permettra de réduire les temps de calculs.

In [11]:
def getPrimeDecomposition(n):
    """Returns a list of all prime factors for a given integer n>0"""
    primes = []
    d = 2
    alpha = 0
    while (n%d==0):
        alpha += 1
        n = int(n/d)
    if (alpha>0):
        primes.append((d, alpha))
    d = 3
    while (d<=n):
        alpha = 0
        while (n%d==0):
            alpha += 1
            n = int(n/d)
        if (alpha>0):
            primes.append((d, alpha))
        d += 2
    return primes

def getDivisorSum(n):
    """Returns the sum of divisors of a given integer n>0"""
    result = 1
    decomposition = getPrimeDecomposition(n)
    for f in decomposition:
        result *= int((f[0]**(f[1]+1)-1)/(f[0]-1))
    return result

In [12]:
getPrimeDecomposition(2)

[(2, 1)]

In [13]:
getPrimeDecomposition(28121964)

[(2, 2), (3, 1), (13, 1), (71, 1), (2539, 1)]

In [14]:
getDivisorSum(196724)-196724

202444

Vérifions que l'on a bien un algorithme plus rapide en recherchant à nouveau une liste de nombres amicaux :

In [15]:
def getFriends2(N):
    """Returns a list of all friendly numbers (m,n) for a given N such that  m<=N"""
    friends = []
    temp = []
    for m in range(2,N+1):
        if not m in temp:
            s1 = getDivisorSum(m)-m
            if (s1>1):
                s2 = getDivisorSum(s1)-s1
                if m==s2 and m!=s1:
                    friends.append((m, s1))
                    temp.append(s1)
    return friends

In [16]:
%time getFriends2(100000)

CPU times: user 53.4 s, sys: 4 ms, total: 53.4 s
Wall time: 53.4 s


[(220, 284),
 (1184, 1210),
 (2620, 2924),
 (5020, 5564),
 (6232, 6368),
 (10744, 10856),
 (12285, 14595),
 (17296, 18416),
 (63020, 76084),
 (66928, 66992),
 (67095, 71145),
 (69615, 87633),
 (79750, 88730)]

In [17]:
%time getFriends2(200000)

CPU times: user 3min 31s, sys: 96 ms, total: 3min 31s
Wall time: 3min 31s


[(220, 284),
 (1184, 1210),
 (2620, 2924),
 (5020, 5564),
 (6232, 6368),
 (10744, 10856),
 (12285, 14595),
 (17296, 18416),
 (63020, 76084),
 (66928, 66992),
 (67095, 71145),
 (69615, 87633),
 (79750, 88730),
 (100485, 124155),
 (122265, 139815),
 (122368, 123152),
 (141664, 153176),
 (142310, 168730),
 (171856, 176336),
 (176272, 180848),
 (185368, 203432),
 (196724, 202444)]

In [18]:
%time getFriends2(300000)

CPU times: user 7min 41s, sys: 156 ms, total: 7min 41s
Wall time: 7min 42s


[(220, 284),
 (1184, 1210),
 (2620, 2924),
 (5020, 5564),
 (6232, 6368),
 (10744, 10856),
 (12285, 14595),
 (17296, 18416),
 (63020, 76084),
 (66928, 66992),
 (67095, 71145),
 (69615, 87633),
 (79750, 88730),
 (100485, 124155),
 (122265, 139815),
 (122368, 123152),
 (141664, 153176),
 (142310, 168730),
 (171856, 176336),
 (176272, 180848),
 (185368, 203432),
 (196724, 202444),
 (280540, 365084)]

In [19]:
def getAliquote(n):
    """ Returns aliquote sequence of integer n>0"""
    aliquote = [n]
    while aliquote.count(n)==1 and n<1000000:
        n = getDivisorSum(n)-n
        if n==0:
            break
        aliquote.append(n)
    return aliquote

def getSocialChains(N):
    """Returns social chains starting with integers less than or equal to N>0"""
    socials = []
    temp = []
    for i in range(1, N+1):
        a = getAliquote(i)
        if set(a) not in temp:
            if len(a[:-1])>2 and 1 not in a and a[-1] == a[0]:
                socials.append(a)
                temp.append(set(a))
    return socials

In [20]:
%time socials = getSocialChains(200000)

CPU times: user 8min 44s, sys: 180 ms, total: 8min 44s
Wall time: 8min 45s


In [21]:
for s in socials:
    print(s)

[12496, 14288, 15472, 14536, 14264, 12496]
[14316, 19116, 31704, 47616, 83328, 177792, 295488, 629072, 589786, 294896, 358336, 418904, 366556, 274924, 275444, 243760, 376736, 381028, 285778, 152990, 122410, 97946, 48976, 45946, 22976, 22744, 19916, 17716, 14316]


Nous avons ainsi exhibé deux chaînes sociables ne faisant intervenir que des nombres inférieurs à 1000000 et contenant au moins un nombre inférieur ou égal à 15000.